# Azure AI Services - ソフト削除済サービス パージユーティリティ

このノートブックは Azure (あるいは他サービス) 上で **ソフト削除状態 (soft-deleted) に残っているサービス/アカウント** を安全に抽出し、保持期間超過したものを対象にパージ (完全削除) するための高度なユーティリティを提供します。

主な特徴:
- 設定値を環境変数/`.env` からロードし安全性と再現性を確保
- ページング対応の全件取得と保持期間ベースのフィルタリング
- Dry-run による事前確認 (誤操作防止)
- 並列/再試行/バックオフ/レート制限対応の堅牢なパージ実行
- 詳細な監査ログと永続化 (CSV/JSON)
- 失敗再試行と最終的な要約統計
- 簡易テストセルで主要ロジックを検証

> ⚠️ パージは不可逆操作です。`DRY_RUN` を必ず最初に `True` のまま動作確認してください。


## 1. 設定値と環境変数の読込

以下の設定値を環境変数または `.env` からロードします。`DRY_RUN=True` で安全に対象確認後、必要なら False に変更してパージします。

| 変数 | 用途 | 必須 | 例 |
|------|------|------|----|
| AZAI_RG | リソースグループ | ✅ | rg-aiservices-demo |
| AZAI_LOCATION | 代表リージョン | ✅ | japaneast |
| AZAI_LOCATIONS | 対象リージョン(カンマ区切り) | 任意 | japaneast,japanwest |
| DRY_RUN | ドライランフラグ | ✅ | True |
| RETENTION_DAYS | 保持期間(日) | ✅ | 7 |
| EXCLUDE_IDS | 除外ID(カンマ区切り) | 任意 | id1,id2 |
| NAME_EXCLUDE_REGEX | 名前除外正規表現 | 任意 | ^test- |
| OUTPUT_DIR | 結果出力ディレクトリ | 任意 | results |
| MAX_WORKERS | 並列ワーカー数 | 任意 | 8 |
| RETRY_LIMIT | 再試行回数 | 任意 | 3 |

## 2. 依存ライブラリ (Azure CLI 前提)
Azure CLI による認証済み (`az login`) を前提とし、Python からは subprocess で `az` を呼び出します。

## 3. Azure CLI ヘルパー
再利用可能な `run_az` 関数を定義し、JSON パースとエラー処理を統一します。

In [3]:
# 1. 環境変数ロード & 設定
import os, re, json, csv, time, datetime, subprocess, shlex
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone, timedelta

# DRY_RUN / RETENTION_DAYS が未設定でも安全にデフォルトを適用 (初回エラー防止)
DEFAULTS = {
  'DRY_RUN': 'true',  # 安全のため true デフォルト
  'RETENTION_DAYS': '7',
  'MAX_WORKERS': '8',
  'RETRY_LIMIT': '3',
  'OUTPUT_DIR': 'results'
}
for k,v in DEFAULTS.items():
    if os.environ.get(k) is None:
        os.environ[k] = v
        print(f'[WARN] env var {k} 未設定のためデフォルト {v} を適用')

def _bool(x):
    return str(x).lower() in ('1','true','yes','on')

def _int(x, default):
    try:
        return int(x)
    except (TypeError,ValueError):
        return default

config = {
    'RESOURCE_GROUP': os.environ.get('AZAI_RG','rg-aiservices-demo'),
    'LOCATION': os.environ.get('AZAI_LOCATION','japaneast'),
    'LOCATIONS': [l.strip() for l in os.environ.get('AZAI_LOCATIONS','').split(',') if l.strip()] or [],
    'DRY_RUN': _bool(os.environ.get('DRY_RUN', DEFAULTS['DRY_RUN'])),
    'RETENTION_DAYS': _int(os.environ.get('RETENTION_DAYS', DEFAULTS['RETENTION_DAYS']),7),
    'EXCLUDE_IDS': [i.strip() for i in os.environ.get('EXCLUDE_IDS','').split(',') if i.strip()],
    'NAME_EXCLUDE_REGEX': os.environ.get('NAME_EXCLUDE_REGEX'),
    'OUTPUT_DIR': os.environ.get('OUTPUT_DIR', DEFAULTS['OUTPUT_DIR']),
    'MAX_WORKERS': _int(os.environ.get('MAX_WORKERS', DEFAULTS['MAX_WORKERS']),8),
    'RETRY_LIMIT': _int(os.environ.get('RETRY_LIMIT', DEFAULTS['RETRY_LIMIT']),3)
}
Path(config['OUTPUT_DIR']).mkdir(parents=True, exist_ok=True)
print('Loaded config:\n', json.dumps(config, ensure_ascii=False, indent=2))

[WARN] env var DRY_RUN 未設定のためデフォルト true を適用
[WARN] env var RETENTION_DAYS 未設定のためデフォルト 7 を適用
[WARN] env var MAX_WORKERS 未設定のためデフォルト 8 を適用
[WARN] env var RETRY_LIMIT 未設定のためデフォルト 3 を適用
[WARN] env var OUTPUT_DIR 未設定のためデフォルト results を適用
Loaded config:
 {
  "RESOURCE_GROUP": "rg-aiservices-demo",
  "LOCATION": "japaneast",
  "LOCATIONS": [],
  "DRY_RUN": true,
  "RETENTION_DAYS": 7,
  "EXCLUDE_IDS": [],
  "NAME_EXCLUDE_REGEX": null,
  "OUTPUT_DIR": "results",
  "MAX_WORKERS": 8,
  "RETRY_LIMIT": 3
}


In [7]:
# 2. 依存ライブラリ (logging 等)
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s %(message)s')
logger = logging.getLogger('purge-cli')
print('Logging ready')

Logging ready


In [8]:
# 3. Azure CLI ヘルパー定義 (存在・ログインチェック含む)
import shutil, subprocess, shlex, json

def _resolve_az():
    for cand in ('az','az.cmd','az.exe'):
        path = shutil.which(cand)
    raise FileNotFoundError(

_IncompleteInputError: incomplete input (702311449.py, line 7)